In [1]:
import torch
import torch.nn as nn
import re
import numpy as np
import json
import os
from sklearn.decomposition import PCA
# from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.utils.tensorboard import SummaryWriter
# from google.colab import drive
# drive.mount('/content/drive')



In [2]:
def slide_split_get_label_from_filename(filename):
    # print(label_mapping.get(filename[9:-5],-1))
    return label_mapping.get(filename[9:-5],-1)

def split_get_label_from_filename(filename):
    return label_mapping.get(filename[8:-5],-1)

def get_label_from_filename(filename):
    match = re.search(r'_(\d+_?\d*)\.json$', filename)
    print(match)
    if match:
        label_key = match.group(1)
        return label_mapping.get(label_key, -1)  # 如果标签不匹配返回 -1

# 获取文件夹中的文件及其标签
def load_files_and_labels(folder_path,split=False):
    file_list = []
    labels = []
    for file in os.listdir(folder_path):
        # print("file: ", file)
        if file.endswith(".json"):
            if split == 1 or split==2 or split==3:
                label = split_get_label_from_filename(file)
            elif split == 4:
                # print(f"split is {split}")
                label = slide_split_get_label_from_filename(file)
            else:
                label = get_label_from_filename(file)
            if label != -1:
                file_list.append(os.path.join(folder_path, file))
                labels.append(label)
            #print(f"File: {file}, Label: {label}")

    return file_list, labels


In [3]:
import json
import numpy as np

# 1. 提取字符并编码为整数索引
def hex_to_sequence(hex_feature):
    """
    将十六进制字符串转换为整数索引序列。
    去掉冒号并转换为字符对应的索引。
    """
    hex_chars = hex_feature.replace(":", "")
    char_to_index = {char: idx for idx, char in enumerate("0123456789abcdef")}
    return [char_to_index[char] for char in hex_chars]

def process_json(file_path):
    """
    处理 JSON 文件，提取时间戳、包长度和 raw data 特征，并对特征进行归一化。
    """
    features = []
    try:
        # 打开 JSON 文件
        with open(file_path, 'r', encoding='utf-8') as file:
            datas = json.load(file)

        # 遍历 JSON 数据，提取时间戳和包长度
        initial_timestamp = None
        pre_time=None
        relative_timestamp = None
        features_1 = []
        features_2 = []
        for data in datas:
            try:
                timestamp = float(data["_source"]["layers"]["frame"]["frame.time_relative"])  # 时间戳
                packet_length = int(data["_source"]["layers"]["frame"]["frame.len"])  # 包长度

                # 获取数据部分，若不存在则为0
                if 'data' in data["_source"]["layers"]:
                    rawdata = data["_source"]["layers"]["data"]["data.data"]
                else:
                    rawdata = '0'

                # 将原始数据转换为整数序列
                data_feature = hex_to_sequence(rawdata)

                # 将数据填充或截断为指定长度 length


                if len(data_feature) < 2832:  # 如果长度小于 2832，填充 0
                    data_feature += [0] * (2832 - len(data_feature))
                elif len(data_feature) > 2832:  # 如果长度大于 2832，截断
                    data_feature = data_feature[:2832]

                # 如果数据长度不符合预期，打印出来
                if len(data_feature) != 2832:
                    print(f"Data feature length mismatch: {len(data_feature)}")

                # 初始化时间戳
                if initial_timestamp is None:
                    initial_timestamp = timestamp  # 设置初始时间戳

                # 计算相对时间戳
                
                relative_timestamp = timestamp - initial_timestamp
                if pre_time==None:
                    pre_time=relative_timestamp
                # print(type(relative_timestamp))
                times_diff=relative_timestamp-pre_time
                pre_time=relative_timestamp



                timestamp_array = np.array([relative_timestamp], dtype=float)
                time_diff_array = np.array([times_diff], dtype=float)
                packet_length_array = np.array([packet_length], dtype=float)/1512#length 归一化
                data_feature = np.array(data_feature, dtype=float)/15

                # 将特征按顺序组合为 [时间戳,time diff, 包长度, ]
                feature_1 = np.hstack((timestamp_array,time_diff_array, packet_length_array))# ,data_feature

                # 添加到特征列表
                features_1.append(feature_1)
                features_2.append(data_feature)

            except (KeyError, ValueError) as e:
                # 跳过有问题的数据包
                print(f"Skipping packet due to error: {e}")
                continue

        features_1_array=np.array(features_1)
        features_2_array=np.array(features_2)
        # print(f"features_1_array.shape",features_1_array.shape)
        # print(f"features_2_array.shape",features_2_array.shape)
        max_timestamp = np.max(features_1_array[:, 0])  # 获取最大时间戳
        # print("max")
        features_1_array[:, 0] = [feature[0] / max_timestamp for feature in features_1_array]  # 时间戳归一化
        max_time_diff=np.max(features_1_array[:, 1])
        features_1_array[:, 1] = [feature[1] / max_time_diff for feature in features_1_array]


        # print("11")

    except Exception as e:
        print(Exception)
        print(f"Error processing file {file_path}:s {e}")
        features_1_array = np.zeros((1, 3))  # 返回空特征以避免程序中断
        features_2_array = np.zeros((1, 2832))

    return features_1_array,features_2_array

In [8]:
split = 0 # 0:100, 1:300, 2:800 3:1000 4:3500
# folder_path = "packet_json_split"
if split==1:
    folder_path = "packet_json_split"
elif split==2:
    folder_path = "202_packet_json_new_800"
elif split==3:
    folder_path = "202_packet_json_new_1000"
elif split==4:
    folder_path = "202_packet_json_new_3500"
else:
    folder_path = "202_packet_json_new"

In [9]:
import nest_asyncio
nest_asyncio.apply()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义文件夹路径 /content/drive/MyDrive/202_project/202_packet_json


# 定义标签映射
label_mapping = {
    "0": 0,             # static
    "0_2": 1,           # slightly_move
    "1": 2,             # move
    "4": 3              # intensely_move
}

# 调用函数获取文件和标签
data_paths, labels = load_files_and_labels(folder_path,split)
# 用于存储每个样本的特征

print(type(labels))
labels_array=np.array(labels)#[:50]
print(labels_array.shape)
print(labels_array)
labels_tensor=torch.tensor(labels_array,dtype=torch.long)

Using device: cuda
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_0.json'>
<re.Match object; span=(5, 12), match='_0.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_

In [10]:

if split==1:fixed_time_steps = 3000
elif split==2: fixed_time_steps = 1600
elif split==3: fixed_time_steps = 1200
elif split==4: fixed_time_steps = 1600
else: fixed_time_steps = 12000
# 截断或补零到固定长度



In [11]:
import numpy as np
import torch
from sklearn.decomposition import PCA

features_1_list = []
features_2_list = []

# 遍历样本，生成 features_1 和 features_2
for data_path in data_paths:
    features_1, features_2 = process_json(data_path)
    features_1_list.append(features_1)
    features_2_list.append(features_2)

print(f"生成了 {len(features_1_list)} 个样本特征")
print(f"第一个样本的特征1形状: {features_1_list[0].shape}")
print(f"第一个样本的特征2形状: {features_2_list[0].shape}")


生成了 100 个样本特征
第一个样本的特征1形状: (13948, 3)
第一个样本的特征2形状: (13948, 2832)


In [12]:

# 对特征2降维
target_dim = 256  # 降到 256 维
pca = PCA(n_components=target_dim)

reduced_features_2_list = []
for feature in features_2_list:
    reduced_feature = pca.fit_transform(feature)  # 对特征2进行PCA降维
    reduced_features_2_list.append(reduced_feature)

# 确保 features_1_list 和 reduced_features_2_list 的时间步一致
for i, (feature1, feature2) in enumerate(zip(features_1_list, reduced_features_2_list)):
    if feature1.shape[0] != feature2.shape[0]:
        raise ValueError(f"样本 {i} 的时间步数不一致：feature1 有 {feature1.shape[0]} 步，降维后的 feature2 有 {feature2.shape[0]} 步")

# 横向拼接特征1和降维后的特征2
combined_features_list = []
for feature1, feature2 in zip(features_1_list, reduced_features_2_list):
    combined_feature = np.hstack((feature1, feature2))  # 横向拼接
    combined_features_list.append(combined_feature)

# 截断或补零到固定时间步
aligned_features = []
for feature in combined_features_list:
    if feature.shape[0] > fixed_time_steps:
        truncated = feature[:fixed_time_steps, :]  # 截断
    else:
        truncated = np.pad(feature, ((0, fixed_time_steps - feature.shape[0]), (0, 0)), mode='constant')  # 补零
    aligned_features.append(truncated)

# 转换为张量
features_tensor = torch.tensor(np.stack(aligned_features, axis=0), dtype=torch.float32)
print(f"截断后的特征张量形状: {features_tensor.shape}")
print(type(features_tensor))

截断后的特征张量形状: torch.Size([100, 12000, 259])
<class 'torch.Tensor'>


In [21]:
if split==1:
    train_dataset=TensorDataset(features_tensor[0:210],labels_tensor[0:210])
    eval_dataset =TensorDataset(features_tensor[210:-1],labels_tensor[210:-1])
elif split==2:
    train_dataset=TensorDataset(features_tensor[0:600],labels_tensor[0:600])
    eval_dataset =TensorDataset(features_tensor[600:-1],labels_tensor[600:-1])
elif split==3:
    train_dataset=TensorDataset(features_tensor[0:600],labels_tensor[0:600])
    eval_dataset =TensorDataset(features_tensor[600:-1],labels_tensor[600:-1])
elif split==4:
    train_dataset=TensorDataset(features_tensor[0:3400],labels_tensor[0:3400])
    eval_dataset =TensorDataset(features_tensor[3400:-1],labels_tensor[3400:-1])
    # 使用 stratified_split 从 3600 数据中抽取 800 样本，分成训练集和验证集
    # train_dataset, eval_dataset = stratified_split(features_tensor, labels_tensor, num_samples_per_class=200, train_ratio=0.75)
else:
    train_dataset=TensorDataset(features_tensor[0:70],labels_tensor[0:70])
    eval_dataset =TensorDataset(features_tensor[70:-1],labels_tensor[70:-1])


batch_size=4
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
eval_loader  = DataLoader(eval_dataset,batch_size=batch_size,shuffle=True)


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2,dropout=0.2):
        super(LSTMModel, self).__init__()

        # Define an LSTM with multiple layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True,dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through LSTM
        out, (hn, cn) = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last time-step's output for classification
        return out


In [22]:
input_size=target_dim+3
hidden_size=16
output_size=4
num_layers = 1     # number of LSTM layers

model = LSTMModel(input_size, hidden_size, output_size, num_layers=num_layers,dropout=0.4)
# model.add(Dense(units))

# inputs=features_tensor
model = model.to(device)
# model = model
# inputs = inputs.to(device)
# labels = labels_tensor
torch.cuda.empty_cache()
# outputs = model(inputs)

# print(f"outputs:{outputs.shape}")

/home/nesl/anaconda3/envs/202_proj/lib/python3.9/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [23]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


True
0
NVIDIA GeForce RTX 4090


In [24]:

# 5. 损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 分类任务使用交叉熵损失
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Decay learning rate every 10 epochs
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)  # Decrease LR by 5% every epoch
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.8,patience=5)
pretrain=True
model_path="LSTM_PCA2_256_hidden16_lr0.001_model_epoch_14tran_acc_78.57% val_acc_48.28%.pth"
if pretrain:
    model.load_state_dict(torch.load(model_path))
    model.to(device)
writer = SummaryWriter('runs/lstm_pca_test')  # This will store logs in 'runs/lstm_training'

# 6. 训练循环
num_epochs = 1000  # 训练轮次
max_accu=0.25
model.eval()
num=0
# Disable gradient computation
with torch.no_grad():
    test_preds = []  # To store predictions
    test_labels = []  # To store true labels
    
    # Loop through the train_loader (or test_loader if you're evaluating the test set)
    for inputs, targets in train_loader:
        # Move inputs and targets to the same device as the model
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Ensure the input shape is (batch_size, seq_len, input_size) for LSTM
        # Adjust this depending on the actual input shape
        # inputs = inputs.view(inputs.size(0), -1, inputs.size(1))  # Assuming inputs have shape (batch_size, input_size)
        
        # Forward pass through the model
        outputs = model(inputs)
        
        # Get predictions by taking the argmax along the output dimension
        _, predicted = torch.max(outputs, 1)
        num+=1
        
        # Store predictions and true labels as NumPy arrays
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(targets.cpu().numpy())
        
        # Print predictions and labels for debugging (optional)
    print("Predictions: ", test_preds)
    print("True Labels: ", test_labels)

    # You may want to calculate accuracy or other metrics after the loop
    # For example, calculate accuracy:

    accuracy = sum(np.array(test_preds) == np.array(test_labels)) / len(test_labels)
    # print(f"Accuracy: {accuracy * 100:.2f}%")
    diff_indices = [i for i in range(len(test_labels)) if test_labels[i] != test_preds[i]]

    print(diff_indices)
    


    # accuracy = accuracy_score(test_labels, test_preds)
    print(f"Test Accuracy: {accuracy*100:.2f}%")

Predictions:  [1, 2, 1, 2, 1, 2, 3, 0, 2, 3, 3, 2, 3, 0, 2, 1, 2, 3, 0, 2, 3, 2, 0, 0, 0, 3, 0, 3, 3, 1, 3, 0, 2, 2, 2, 0, 0, 3, 1, 3, 1, 2, 0, 3, 0, 1, 2, 0, 0, 3, 0, 3, 3, 0, 2, 1, 2, 2, 2, 3, 0, 3, 0, 2, 3, 3, 2, 3, 0, 0]
True Labels:  [1, 0, 0, 3, 1, 2, 1, 3, 2, 0, 1, 1, 3, 0, 0, 0, 3, 1, 1, 3, 3, 3, 1, 0, 0, 1, 3, 0, 1, 0, 2, 0, 3, 2, 2, 1, 3, 1, 1, 3, 1, 2, 0, 3, 0, 0, 3, 3, 0, 2, 2, 3, 3, 2, 2, 3, 2, 0, 2, 2, 0, 3, 2, 2, 0, 1, 2, 2, 2, 3]
[1, 2, 3, 6, 7, 9, 10, 11, 14, 15, 16, 17, 18, 19, 21, 22, 25, 26, 27, 28, 29, 30, 32, 35, 36, 37, 45, 46, 47, 49, 50, 53, 55, 57, 59, 62, 64, 65, 67, 68, 69]
Test Accuracy: 41.43%
